In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
from pyspark import SparkConf,SparkContext
import matplotlib.pyplot as plt
from time import time 
from tqdm import tqdm
import sklearn

from pyspark.sql.functions import col, countDistinct

In [2]:
from pyspark.sql import SparkSession

In [3]:
sc = SparkSession.builder.appName('yelp-data-exploration').getOrCreate()

In [4]:
yelp_users = sc.read.json("/home/bh_parijat/data/yelp/user.json")

In [5]:
yelp_reviews = sc.read.json("/home/bh_parijat/data/yelp/review.json")

In [6]:
yelp_business = sc.read.json("/home/bh_parijat/data/yelp/business.json")

In [34]:
yelp_tip = sc.read.json("/home/bh_parijat/data/yelp/tip.json")

In [7]:
yelp_reviews.count()

6685900

In [8]:
reviews_columns = ["business_id", "cool","date","funny","review_id","stars","text","useful","user_id"]

In [9]:
yelp_reviews_df = yelp_reviews.toDF(*reviews_columns)

In [16]:
yelp_reviews.head(1)

[Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA')]

In [11]:
yelp_reviews_df.head()

Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA')

In [17]:
yelp_users.head(1)

[Row(average_stars=4.03, compliment_cool=1, compliment_cute=0, compliment_funny=1, compliment_hot=2, compliment_list=0, compliment_more=0, compliment_note=1, compliment_photos=0, compliment_plain=1, compliment_profile=0, compliment_writer=2, cool=25, elite='2015,2016,2017', fans=5, friends='c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvzqt

In [13]:
yelp_users.count()

1637138

In [14]:
yelp_users.columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [18]:
users_columns = yelp_users.columns

In [54]:
users_columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [26]:
yelp_users_df = yelp_users.toDF(*users_columns)

In [ ]:
yelp_users_df.collect()

In [20]:
business_columns = yelp_business.columns

In [56]:
yelp_business.head(1)

[Row(address='2818 E Camino Acequia Drive', attributes=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=None, Ambience=None, BYOB=None, BYOBCorkage=None, BestNights=None, BikeParking=None, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=None, BusinessParking=None, ByAppointmentOnly=None, Caters=None, CoatCheck=None, Corkage=None, DietaryRestrictions=None, DogsAllowed=None, DriveThru=None, GoodForDancing=None, GoodForKids='False', GoodForMeal=None, HairSpecializesIn=None, HappyHour=None, HasTV=None, Music=None, NoiseLevel=None, Open24Hours=None, OutdoorSeating=None, RestaurantsAttire=None, RestaurantsCounterService=None, RestaurantsDelivery=None, RestaurantsGoodForGroups=None, RestaurantsPriceRange2=None, RestaurantsReservations=None, RestaurantsTableService=None, RestaurantsTakeOut=None, Smoking=None, WheelchairAccessible=None, WiFi=None), business_id='1SWheh84yJXfytovILXOAQ', categories='Golf, Active Life', city='Phoenix', hours=None, is_open=0, latitude=33.5221425, longit

In [21]:
business_columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [22]:
yelp_checkin = sc.read.json("/home/bh_parijat/data/yelp/checkin.json")

In [23]:
checkin_columns = yelp_checkin.columns

In [24]:
checkin_columns

['business_id', 'date']

In [26]:
yelp_checkin.head(1)

[Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')]

In [29]:
yelp_checkin.count()

161950

In [28]:
yelp_business.agg

DataFrame[address: string, attributes: struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>, business_id: string, categories: string, city: string, hours: struct<Friday:string,Monday:string,Saturday:string

In [31]:
yelp_business.agg(countDistinct(col("business_id")).alias("different_business")).show()

+------------------+
|different_business|
+------------------+
|            192609|
+------------------+



In [32]:
yelp_checkin.agg(countDistinct(col("business_id")).alias("different_business")).show()

+------------------+
|different_business|
+------------------+
|            161950|
+------------------+



In [33]:
yelp_business.count()

192609

In [35]:
yelp_tip

DataFrame[business_id: string, compliment_count: bigint, date: string, text: string, user_id: string]

In [36]:
yelp_tip.head(1)

[Row(business_id='VaKXUpmWTTWDKbpJ3aQdMw', compliment_count=0, date='2014-03-27 03:51:24', text='Great for watching games, ufc, and whatever else tickles yer fancy', user_id='UPw5DWs_b-e2JRBS-t37Ag')]

In [37]:
yelp_tip.count()

1223094

In [38]:
yelp_tip.agg(countDistinct(col("user_id")).alias("different_users")).show()

+---------------+
|different_users|
+---------------+
|         320685|
+---------------+



In [40]:
yelp_tip.agg(countDistinct(col("business_id")).alias("business_id")).show()

+-----------+
|business_id|
+-----------+
|     123997|
+-----------+



In [52]:
yelp_tip.groupBy("user_id")\
        .agg({'compliment_count':'count'})\
        .withColumnRenamed("count(compliment_count)","count")\
        .sort(col("count").desc()).show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|mkbx55W8B8aPLgDqe...| 2439|
|CxDOIDnH8gp9KXzpB...| 1598|
|6ZC-0LfOAGwaFc5XP...| 1509|
|0tvCcnfJnSs55iB6m...| 1376|
|eZfHm0qI8A_HfvXSc...| 1352|
|O8eDScRAg6ae0l9Bc...| 1255|
|8DGFWco9VeBAxjqsu...| 1178|
|WJKocp9RE0KatUwh3...| 1161|
|2EuPAGalYnP7eSxPg...| 1154|
|QPJJohtGqkMkaN0Gt...| 1017|
|3nDUQBjKyVor5wV0r...| 1002|
|5dKknvq65x-SaluuJ...|  997|
|A0j21z2Q1HGic7jW6...|  986|
|0FMte0z-repSVWSJ_...|  925|
|uG35h72BAMutvXAWd...|  846|
|RQlnSCjuqMnhR3Qk6...|  820|
|tQPk4JiBPsx7NSIDb...|  794|
|Fw4UjJ6yBeyPB27Y4...|  793|
|fmzIm7RxEdii5Jz44...|  773|
|pn_flI3EBNugBEYFp...|  771|
+--------------------+-----+
only showing top 20 rows



In [53]:
yelp_tip.groupBy("business_id")\
        .agg({'compliment_count':'count'})\
        .withColumnRenamed("count(compliment_count)","count")\
        .sort(col("count").desc()).show()

+--------------------+-----+
|         business_id|count|
+--------------------+-----+
|FaHADZARwnY4yvlvp...| 3614|
|JmI9nslLD7KZqRr__...| 2440|
|DkYS3arLOhA8si5uU...| 1503|
|5LNZ67Yw9RD6nf4_U...| 1487|
|K7lWdNUhCbcnEvI0N...| 1386|
|hihud--QRriCYZw1z...| 1371|
|RESDUcs7fIiihp38-...| 1264|
|4JNXUYY8wbaaDmk3B...| 1121|
|yfxDa8RFOvJPQh0rN...| 1117|
|iCQpiavjjPzJ5_3gP...| 1105|
|SMPbvZLSMMb7KU76Y...| 1025|
|7sPNbCx7vGAaH7SbN...| 1018|
|UPIYuRaZvknINOd1w...| 1013|
|eoHdUeQDNgQ6WYEnP...|  954|
|yQab5dxZzgBLTEHCw...|  938|
|LNGBEEelQx4zbfWnl...|  904|
|JyxHvtj-syke7m9rb...|  903|
|f4x1YBxkLrZg652xt...|  860|
|WUq8HJHIZU4uteB15...|  846|
|Wxxvi3LZbHNIDwJ-Z...|  792|
+--------------------+-----+
only showing top 20 rows



In [55]:
yelp_reviews

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [83]:
yelp_reviews.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [79]:
r = yelp_reviews.select("text").limit(2).collect()

In [81]:
r[1]

Row(text="I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair has ever seen.  The team of stylists clearly gets along extremely well, as it's evident from the way they talk to and help one another that it's really genuine and not some corporate requirement.  It 

In [84]:
yelp_reviews.count()

6685900

In [ ]:
yelp_reviews.groupBy("stars")
            .agg({"text":"count"})
            .withColumnRenamed("count(text)",count)
            .sort(col("count").desc())
            .s